In [1]:
import numpy as np
import pandas as pd
import json
import pickle
import matplotlib.pyplot as plt
import warnings

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch.utils.data

import matplotlib.pyplot as plt
import os

In [2]:
'''
Read in saved dataset to use to feed into the model.
Further pre-processing still needs to be done on the state vectors
'''

data_df = pd.read_pickle('/home/amans/Development/scott/nfl-big-data-bowl-2022/clean_dir/from_csv.pkl')

data_df.head()

,season,gameId,playId,playIndex,timeIndex,time,football_pos,state,next_state,reward,action,next_action
0,2018,2018090600,677,0,48,2018-09-07T01:34:34.100,"[[12.42, 42.51, 6.89, 7.5, 0.0, 0.0]]","[[11.4, 42.67, 0.2, 3.07, 102.1, 312.17, 88.6,...","[[11.39, 42.66, 0.21, 2.84, 105.48, 199.45, 88...",-0.01,"[0.01, 0.01, -0.01, 0.23, -3.38, 112.72, -0.01...","[-0.02, 0.03, -0.26, 0.31, -8.38, 31.09, 0.02,..."
1,2018,2018090600,677,0,49,2018-09-07T01:34:34.200,"[[11.58, 42.54, 6.16, 6.87, 0.0, 0.0]]","[[11.39, 42.66, 0.21, 2.84, 105.48, 199.45, 88...","[[11.41, 42.63, 0.47, 2.53, 113.86, 168.36, 88...",0.02,"[-0.02, 0.03, -0.26, 0.31, -8.38, 31.09, 0.02,...","[-0.04, 0.07, -0.36, 0.0, 3.17, 10.04, 0.04, 0..."
2,2018,2018090600,677,0,50,2018-09-07T01:34:34.300,"[[11.3, 42.45, 4.94, 7.15, 0.0, 0.0]]","[[11.41, 42.63, 0.47, 2.53, 113.86, 168.36, 88...","[[11.45, 42.56, 0.83, 2.53, 110.69, 158.32, 88...",0.04,"[-0.04, 0.07, -0.36, 0.0, 3.17, 10.04, 0.04, 0...","[-0.08, 0.11, -0.49, -0.5, -6.61, 10.26, 0.08,..."
3,2018,2018090600,677,0,51,2018-09-07T01:34:34.400,"[[11.32, 42.34, 3.78, 7.54, 0.0, 0.0]]","[[11.45, 42.56, 0.83, 2.53, 110.69, 158.32, 88...","[[11.53, 42.45, 1.32, 3.03, 117.3, 148.06, 88....",0.08,"[-0.08, 0.11, -0.49, -0.5, -6.61, 10.26, 0.08,...","[-0.09, 0.13, -0.38, -0.1, 2.88, 3.22, 0.09, 0..."
4,2018,2018090600,677,0,52,2018-09-07T01:34:34.500,"[[11.15, 42.28, 2.91, 6.78, 0.0, 0.0]]","[[11.53, 42.45, 1.32, 3.03, 117.3, 148.06, 88....","[[11.62, 42.32, 1.7, 3.13, 114.42, 144.84, 88....",0.09,"[-0.09, 0.13, -0.38, -0.1, 2.88, 3.22, 0.09, 0...","[-0.16, 0.16, -0.53, -0.24, -17.77, 7.48, 0.16..."


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cpu


/home/amans/.local/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
'''
Create torch dataset
'''

features = torch.zeros(len(data_df)).to(device)
labels = torch.zeros(len(data_df)).to(device)

print("size = ", features.size())

size =  torch.Size([209239])


In [5]:
'''
Function that splits the data into a training, validation, and test set
'''
def split_data(dataset, train_split, seed):
    np.random.seed(seed)
    indices = list(range(len(dataset)))
    np.random.shuffle(indices)

    train_num = int(len(dataset)*train_split)
    val_num = (len(dataset) - int(len(dataset)*train_split))//2

    train_indices = indices[0:train_num]
    val_indices = indices[train_num:train_num+val_num]
    test_indices = indices[train_num+val_num:]

    #check to make sure slices correct
    assert len(dataset) == len(train_indices) + len(val_indices) + len(test_indices)

    #dataset = help.normalize(train_indices, dataset)

    train_data = dataset.iloc[train_indices,:]
    val_data = dataset.iloc[val_indices,:]
    test_data = dataset.iloc[test_indices,:]

    return train_data, val_data, test_data

train_data, val_data, test_data = split_data(data_df, 0.7, 2430)

print(test_data.columns)

print(f"Length of training data: {len(train_data)}")

Index(['season', 'gameId', 'playId', 'playIndex', 'timeIndex', 'time',
       'football_pos', 'state', 'next_state', 'reward', 'action',
       'next_action'],
      dtype='object')
Length of training data: 146467


In [6]:
'''
Define models
'''

'''
Define hyperparamters
'''
BATCH_SIZE = 128
state_size = len(data_df.loc[0,'state'])
action_size = 4
gamma = 0.99
#max_action = torch.tensor(max_action, dtype=torch.float32).to(device=device)

print(f"State size: {state_size}")
print(f"Action size: {action_size}")
    
'''
First CNN model
cnn_model = nn.Sequential(
    nn.Conv2d(1, 12, (12,1), stride=1, padding=0),
    nn.LeakyReLU(),
    nn.Conv2d(12, 24, (6, 1)),
    nn.LeakyReLU(),
    nn.Conv2d(24, 48, (1,1)),
    nn.Flatten(),
    nn.Linear(48*6*6,action_size)
)

cnn_model.to(device=device)
'''

State size: 22
Action size: 4


'\nFirst CNN model\ncnn_model = nn.Sequential(\n    nn.Conv2d(1, 12, (12,1), stride=1, padding=0),\n    nn.LeakyReLU(),\n    nn.Conv2d(12, 24, (6, 1)),\n    nn.LeakyReLU(),\n    nn.Conv2d(24, 48, (1,1)),\n    nn.Flatten(),\n    nn.Linear(48*6*6,action_size)\n)\n\ncnn_model.to(device=device)\n'

In [7]:
'''
cnn_model = nn.Sequential(
    nn.Conv2d(1, 32, (3,3), stride=1,padding=0),
    nn.LeakyReLU(),
    #nn.MaxPool2d((2,2), stride=2),
    nn.Conv2d(32, 64, (3,3), stride=1,padding=0),
    nn.LeakyReLU(),
    #nn.MaxPool2d((2,2), stride=2),
    nn.Conv2d(64,128, (2,2), stride=1,padding=0),
    nn.LeakyReLU(),
    #nn.MaxPool2d((2,2), stride=2),
    nn.Conv2d(128,256, (1,1), stride=1,padding=0),
    #nn.LeakyReLU(),
    #nn.MaxPool2d((2,2), stride=2),
    #nn.Conv2d(256,512, (3,3), stride=1,padding=0),
    nn.LeakyReLU(),
    #nn.MaxPool2d((2,2), stride=2),
    nn.Flatten(),
    nn.Linear(4352, 2000),
    nn.LeakyReLU(),
    nn.Linear(2000, 1000),
    nn.LeakyReLU(),
    nn.Linear(1000, 100),
    nn.LeakyReLU(),
    nn.Linear(100, action_size)
)
cnn_model.to(device=device)

cnn_model = nn.Sequential(
    nn.Conv2d(1, 4, (2,1), stride=1, padding=0),
    nn.LeakyReLU(),
    nn.Flatten(),
    nn.Linear(504,200), 
    nn.LeakyReLU(),
    nn.Linear(200,action_size)
)

cnn_model.to(device=device)
'''

cnn_model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(88,50), 
    nn.LeakyReLU(),
    nn.Linear(50,25),
    nn.LeakyReLU(),
    nn.Linear(25,action_size)
)

cnn_model.to(device=device)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=88, out_features=50, bias=True)
  (2): LeakyReLU(negative_slope=0.01)
  (3): Linear(in_features=50, out_features=25, bias=True)
  (4): LeakyReLU(negative_slope=0.01)
  (5): Linear(in_features=25, out_features=4, bias=True)
)

In [8]:
# define loss function
mse_loss_fn = nn.MSELoss()
# define optimizers
cnn_optimizer = optim.Adam(cnn_model.parameters())

In [9]:
z = np.zeros((1,1,2,4))

z[:,:,:,[0,2,3]]

array([[[[0., 0., 0.],
         [0., 0., 0.]]]])

In [10]:
'''
Given indices, train model on batch
'''
def train_loop(df, indices, model, loss_fn, optimizer, device):
    size = len(indices)
    play = df.iloc[indices, :]
    
    numpy_state = np.stack(play['state'].values)
    batch_size = numpy_state.shape[0]
    numpy_state_input = numpy_state.reshape(batch_size, 1, -1, 8)[:,:,:,[2,3,6,7]]

    true_state = torch.tensor(numpy_state, dtype=torch.float32).to(device=device)
    true_action = torch.tensor(np.stack(play['action'].values)[:,[2,3,6,7]], dtype=torch.float32).to(device=device)
    
    # shape true state into matrix to pass into CNN
    #true_state_mod = true_state.reshape(batch_size, -1, len(true_action))
    true_state_mod = torch.tensor(numpy_state_input, dtype=torch.float32).to(device=device)

    model.train()  # put model to training mode

    #compute prediction and loss
    predicted_action = model(true_state_mod)
    
    loss = loss_fn(predicted_action, true_action)

    #Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss/size

In [11]:
def test_loop(test_df, model, loss_fn, device):
    size = len(test_df)
    test_loss = 0
    
    with torch.no_grad():
        for row_index in range(0,len(test_df)):

            play = test_df.iloc[row_index, :]

            true_state = torch.tensor(play['state'], dtype=torch.float32).to(device=device)
            true_action = torch.tensor(play['action'][[2,3,6,7]], dtype=torch.float32).to(device=device)
            
            # useful for if testing in batches
            #true_state = torch.tensor(np.stack(play['state'].values), dtype=torch.float32).to(device=device)
            #true_action = torch.tensor(np.stack(play['action'].values), dtype=torch.float32).to(device=device)

            numpy_state_input = play['state'].reshape(1, 1, -1, 8)[:,:,:,[2,3,6,7]]
                           
            true_state_mod = torch.tensor(numpy_state_input, dtype=torch.float32).to(device=device)

            model.eval()  # put model to eval mode

            #compute prediction and loss
            predicted_action = model(true_state_mod)
  
            true_action = torch.unsqueeze(true_action, 0)
            
            loss = loss_fn(predicted_action, true_action)
            
            test_loss += loss
        
        test_loss = test_loss/size

    
    return test_loss

In [12]:
import random

epochs = 5
break_var = False
training_loss_list = []
val_loss_list = []

num_batches = int(len(train_data)/BATCH_SIZE)+1

print(f"Num batches: {num_batches}")

indexes = list(range(0,len(train_data)))

np.random.seed(2430)

for k in range(epochs):
    
    random.shuffle(indexes)
    
    for i in range(num_batches):

        start_index = i*BATCH_SIZE
        end_index = min(len(train_data), (i+1)*BATCH_SIZE)
        
        #indices = list(range(start_index, end_index))
        indices = indexes[start_index:end_index]

        train_loss = train_loop(train_data, indices, cnn_model, mse_loss_fn, cnn_optimizer, device)
        
        training_loss_list.append(train_loss)

        if (i % int(num_batches/3) == 0): #and (i != 0):
            val_loss = test_loop(val_data, cnn_model, F.mse_loss, device)
            val_loss_list.append(val_loss)
            print(f"At epoch {k}, iter {i}: train loss = {train_loss}")
            print(f"At epoch {k}, iter {i}: val loss = {val_loss}")
            
torch.save(cnn_model.state_dict(), 'cnn2.pt')

Num batches: 1145
At epoch 0, iter 0: train loss = 0.059754058718681335
At epoch 0, iter 0: val loss = 4.0469512939453125
At epoch 0, iter 381: train loss = 0.0011124168522655964
At epoch 0, iter 381: val loss = 0.12206614762544632
At epoch 0, iter 762: train loss = 0.0009948015213012695
At epoch 0, iter 762: val loss = 0.11457198858261108
At epoch 0, iter 1143: train loss = 0.0008711127447895706
At epoch 0, iter 1143: val loss = 0.10843068361282349
At epoch 1, iter 0: train loss = 0.0009675890323705971
At epoch 1, iter 0: val loss = 0.11415652930736542
At epoch 1, iter 381: train loss = 0.000893519027158618
At epoch 1, iter 381: val loss = 0.10634426027536392
At epoch 1, iter 762: train loss = 0.0008622316527180374
At epoch 1, iter 762: val loss = 0.10120908915996552
At epoch 1, iter 1143: train loss = 0.0007408352103084326
At epoch 1, iter 1143: val loss = 0.09907961636781693
At epoch 2, iter 0: train loss = 0.0007659094990231097
At epoch 2, iter 0: val loss = 0.10080092400312424
At 